In [1]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
import qdrant_client.http.models as qmodels
from config_qdrant import *
from qdrant_client import QdrantClient
from pprint import pprint
from sentence_transformers import CrossEncoder

c:\Ambarish\JUWORKSHOP_JAN_2026\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
qdrant_client = QdrantClient(
    url=HOST,
    api_key=API_KEY
)

In [3]:
# Get the user question
user_question = "cute gray fuzzy bee"
COLLECTION_NAME ="BEES"

In [4]:
project = AIProjectClient(
    endpoint=AZURE_OPENAI_ENDPOINT,
    credential=DefaultAzureCredential(),
)

client = project.get_openai_client(api_version="2024-10-21")

MODEL_NAME = AZURE_OPENAI_DEPLOYMENT_ID

### RAG without Reranking

In [5]:
query_vector = client.embeddings.create(model="text-embedding-3-small", input=user_question).data[0].embedding

In [6]:
search_result = qdrant_client.query_points(collection_name=COLLECTION_NAME,
                                        query=query_vector, 
                                        query_filter=None,
                                        limit=5)

In [7]:
search_result

QueryResponse(points=[ScoredPoint(id='15ff5ef2-f115-40aa-85bc-575cce731913', version=3, score=0.4901175, payload={'sourcefile': '../RAG/data/Centris_pallida.pdf', 'content': '## **Description and identification**\n\n\nThis bee is black and densely covered in a grey pubescence or\nfur on the [dorsal](https://en.wikipedia.org/wiki/Anatomical_terms_of_location) side. The [thorax](https://en.wikipedia.org/wiki/Thorax_(insect_anatomy)) fur has a slightly yellow\ncolor. The legs have a mixture of black and reddish fur. The\n[ventral side of the bee is covered in a brownish or dark yellow](https://en.wikipedia.org/wiki/Anatomical_terms_of_location)\nfur. The wings are fairly transparent except for the black veins\nthat run through them. Males and females are similar in size at\nabout 16–17 millimeters. Males have eyes more yellow in\ncolor, and their thorax fur is lighter. Females have eyes more\ngreen in color, and their thorax fur is more brown than grey. [[3]]\n\n## **Distribution and habi

In [8]:
context = ""

In [9]:
for scored_point in search_result.points:
    context = context + " " + (scored_point.payload["content"])

In [10]:
len(context)

6898

In [11]:
# Now we can use the matches to generate a response
SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about insects.
You must use the data set to answer the questions,
you should not provide any info that is not in the provided sources.
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.3,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": f"{user_question}\nSources: {context}"},
    ],
)

print(f"\nResponse from {MODEL_NAME} on : \n")
pprint(response.choices[0].message.content)


Response from gpt-4o-mini on : 

('The cute gray fuzzy bee you are referring to is likely _Centris pallida_. '
 'This bee is characterized by its black body densely covered in gray fur on '
 'the dorsal side, with slightly yellow fur on the thorax. The legs feature a '
 'mix of black and reddish fur, while the ventral side is covered in brownish '
 'or dark yellow fur. Males and females are similar in size, measuring about '
 '16–17 millimeters, but differ in eye color and thorax fur coloration.\n'
 '\n'
 '_Centris pallida_ is commonly found in dry, hot environments across North '
 'America, particularly in Arizona, Nevada, southern California, New Mexico, '
 'and western Mexico. They are well-adapted to survive cold nights in the '
 'desert and are mostly inactive during the day, seeking shade or burrows to '
 'avoid overheating.')


### ReRank

In [12]:
def rerank(query, retrieved_documents):
    """
    Rerank the results using a cross-encoder model.
    """
    encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
    scores = encoder.predict([(query, doc.payload["content"]) for doc in retrieved_documents])
    scored_documents = [v for _, v in sorted(zip(scores, retrieved_documents), reverse=True)]
    return scored_documents

In [13]:
reranked_results = rerank(user_question, search_result.points)

Loading weights: 100%|██████████| 105/105 [00:00<00:00, 211.66it/s, Materializing param=classifier.weight]                                    
BertForSequenceClassification LOAD REPORT from: cross-encoder/ms-marco-MiniLM-L-6-v2
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [14]:
reranked_results

[ScoredPoint(id='15ff5ef2-f115-40aa-85bc-575cce731913', version=3, score=0.4901175, payload={'sourcefile': '../RAG/data/Centris_pallida.pdf', 'content': '## **Description and identification**\n\n\nThis bee is black and densely covered in a grey pubescence or\nfur on the [dorsal](https://en.wikipedia.org/wiki/Anatomical_terms_of_location) side. The [thorax](https://en.wikipedia.org/wiki/Thorax_(insect_anatomy)) fur has a slightly yellow\ncolor. The legs have a mixture of black and reddish fur. The\n[ventral side of the bee is covered in a brownish or dark yellow](https://en.wikipedia.org/wiki/Anatomical_terms_of_location)\nfur. The wings are fairly transparent except for the black veins\nthat run through them. Males and females are similar in size at\nabout 16–17 millimeters. Males have eyes more yellow in\ncolor, and their thorax fur is lighter. Females have eyes more\ngreen in color, and their thorax fur is more brown than grey. [[3]]\n\n## **Distribution and habitat**\n\n\n_Centris p

In [15]:
context =""
for scored_point in reranked_results:
    context = context + " " + (scored_point.payload["content"])

In [16]:
context

' ## **Description and identification**\n\n\nThis bee is black and densely covered in a grey pubescence or\nfur on the [dorsal](https://en.wikipedia.org/wiki/Anatomical_terms_of_location) side. The [thorax](https://en.wikipedia.org/wiki/Thorax_(insect_anatomy)) fur has a slightly yellow\ncolor. The legs have a mixture of black and reddish fur. The\n[ventral side of the bee is covered in a brownish or dark yellow](https://en.wikipedia.org/wiki/Anatomical_terms_of_location)\nfur. The wings are fairly transparent except for the black veins\nthat run through them. Males and females are similar in size at\nabout 16–17 millimeters. Males have eyes more yellow in\ncolor, and their thorax fur is lighter. Females have eyes more\ngreen in color, and their thorax fur is more brown than grey. [[3]]\n\n## **Distribution and habitat**\n\n\n_Centris pallida_ are located in dry, hot environments of North\nAmerica. Specifically, they are in Arizona, Nevada, southern\nCalifornia, New Mexico, and western

In [17]:
# Now we can use the matches to generate a response
SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about insects.
You must use the data set to answer the questions,
you should not provide any info that is not in the provided sources.
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.3,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": f"{user_question}\nSources: {context}"},
    ],
)

print(f"\nResponse from {MODEL_NAME} on : \n")
pprint(response.choices[0].message.content)


Response from gpt-4o-mini on : 

('The cute gray fuzzy bee you are referring to is likely _Centris pallida_. '
 'This bee is characterized by its black body densely covered in gray fur, '
 'with slightly yellow fur on the thorax. The legs have a mix of black and '
 'reddish fur, while the ventral side is covered in brownish or dark yellow '
 'fur. Males and females are similar in size, measuring about 16–17 '
 'millimeters, but can be distinguished by the color of their eyes and the '
 'shade of their thorax fur. Males have yellow eyes and lighter thorax fur, '
 'whereas females have green eyes and a thorax that is more brown than gray. \n'
 '\n'
 '_Centris pallida_ is commonly found in dry, hot environments across North '
 'America, particularly in Arizona, Nevada, southern California, New Mexico, '
 'and western Mexico. They are quite common, especially in Arizona, and are '
 'classified as Least Concern in terms of conservation. Their fur and dark '
 'exoskeleton help them survive 